# **Lab 1: AI‑Assisted SQL Foundations (Colab)**
**Course:** MGMT 467 — AI‑Assisted Big Data Analytics in the Cloud  
**When:** Unit 1 · Week 3 (Thursday)  
**Goal:** Use *Gemini as a co‑pilot* to write and understand basic SQL against the **Superstore** dataset in **BigQuery**.  
**You will practice:** Prompt engineering → SQL generation → Query execution → Interpretation.

> ✅ **Deliverable (submit link in Brightspace):** A completed `Lab1_AI_Assisted_SQL.ipynb` with prompts, code, outputs.


## ✅ What you need before starting
- A Google account with access to **Google Colab**.
- A **Google Cloud Project** with **BigQuery** API enabled.
- A dataset named `superstore_data` and a table named `sales` (your instructor or TA will provide details).
- (Optional) A GitHub repo to store your work for your team.


## 0) Connect to class GitHub repo
## Install and import libraries
## Class Repo: bigDataNCloud/mgmt_467_resources

In [1]:
# Replace with your actual class repo URL
CLASS_REPO_URL = "https://github.com/cruz209/MGMT467"
CLASS_REPO_DIR = "/content/class-repo"

# If already cloned, skip re-downloading
import os
if not os.path.exists(CLASS_REPO_DIR):
    !git clone {CLASS_REPO_URL} {CLASS_REPO_DIR}
else:
    %cd {CLASS_REPO_DIR}
    !git pull

# Move into repo folder
%cd {CLASS_REPO_DIR}

print("✅ Class repository ready at", CLASS_REPO_DIR)

Cloning into '/content/class-repo'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (5/5), 6.73 KiB | 1.35 MiB/s, done.
/content/class-repo
✅ Class repository ready at /content/class-repo


In [2]:
!pip -q install google-cloud-bigquery google-cloud-bigquery-storage db-dtypes pandas pyarrow kagglehub

In [3]:
from google.cloud import bigquery
from google.colab import auth
import pandas as pd
import os, json, textwrap, pathlib, pprint
print("Libraries imported.")

Libraries imported.


## 1) Authenticate to GCP and set your project
Run the cell below. When prompted, authorize Colab to access your Google Cloud resources.

In [5]:
auth.authenticate_user()
print("✅ Authenticated.")

# Set your GCP project (EDIT THIS!)
PROJECT_ID = "imposing-coast-442802-a7"   # <-- EDIT ME
assert PROJECT_ID != "YOUR_PROJECT_ID_HERE", "Please set PROJECT_ID to your GCP project id."
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

# BigQuery client
bq = bigquery.Client(project=PROJECT_ID)
print("✅ BigQuery client created for:", PROJECT_ID)

✅ Authenticated.
✅ BigQuery client created for: imposing-coast-442802-a7


### Quick connection test (optional)
This will try to read the first 5 rows from your Superstore table. If it fails, double‑check your dataset/table name.

In [8]:
DATASET = "superstore_data"
TABLE   = "sales"
BQ_TABLE = f"`{PROJECT_ID}.{DATASET}.{TABLE}`"

query = f"SELECT * FROM {BQ_TABLE} LIMIT 5"
try:
    df_preview = bq.query(query).to_dataframe()
    display(df_preview)
    print("✅ Connection OK.")
except Exception as e:
    print("⚠️ Could not query the table. Error below:")
    print(e)

⚠️ Could not query the table. Error below:
404 Not found: Table imposing-coast-442802-a7:superstore_data.sales was not found in location US; reason: notFound, message: Not found: Table imposing-coast-442802-a7:superstore_data.sales was not found in location US

Location: US
Job ID: 736475b3-90c1-46a9-ba1e-76c2bfaddaa3



In [10]:
# Assuming the "superstore-dataset-final" dataset downloaded from Kaggle is in the download_path directory
# and contains a file named "Sample - Superstore.csv" (adjust filename if needed)

dataset_id = DATASET  # Reuse the dataset name from the failed cell
table_id = TABLE # Reuse the table name from the failed cell

# Construct a BigQuery client object.
client = bigquery.Client()

# Configure the load job
job_config = bigquery.LoadJobConfig(
    schema=[
        # Define schema based on your CSV file - this is a simplified example
        # You might need to inspect the CSV file to get the correct schema
        bigquery.SchemaField("Row_ID", "INTEGER"),
        bigquery.SchemaField("Order_ID", "STRING"),
        bigquery.SchemaField("Order_Date", "STRING"),
        bigquery.SchemaField("Ship_Date", "STRING"),
        bigquery.SchemaField("Ship_Mode", "STRING"),
        bigquery.SchemaField("Customer_ID", "STRING"),
        bigquery.SchemaField("Customer_Name", "STRING"),
        bigquery.SchemaField("Segment", "STRING"),
        bigquery.SchemaField("Country", "STRING"),
        bigquery.SchemaField("City", "STRING"),
        bigquery.SchemaField("State", "STRING"),
        bigquery.SchemaField("Postal_Code", "INTEGER"),
        bigquery.SchemaField("Region", "STRING"),
        bigquery.SchemaField("Product_ID", "STRING"),
        bigquery.SchemaField("Category", "STRING"),
        bigquery.SchemaField("Sub_Category", "STRING"),
        bigquery.SchemaField("Product_Name", "STRING"),
        bigquery.SchemaField("Sales", "FLOAT"),
        bigquery.SchemaField("Quantity", "INTEGER"),
        bigquery.SchemaField("Discount", "FLOAT"),
        bigquery.SchemaField("Profit", "FLOAT"),
    ],
    skip_leading_rows=1,  # Adjust if your CSV has a header row
    source_format=bigquery.SourceFormat.CSV,
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE # Overwrite the table if it exists
)

# Get the path to the downloaded CSV file
file_path = download_path + "/Sample - Superstore.csv" # Adjust filename if needed

with open(file_path, "rb") as source_file:
    job = client.load_table_from_file(source_file, f"{PROJECT_ID}.{dataset_id}.{table_id}", job_config=job_config)  # Make an API request.

job.result()  # Wait for the job to complete.

print(f"Loaded {job.output_rows} rows into {PROJECT_ID}.{dataset_id}.{table_id}")

Loaded 9994 rows into imposing-coast-442802-a7.superstore_data.sales


## 2) (Optional) Download a dataset with **KaggleHub**
Use **KaggleHub** to pull sample data locally into Colab (e.g., for offline exploration or to stage for GCS).
> This is **optional** for Lab 1 (we focus on BigQuery), but useful to practice data access workflows.
> https://www.kaggle.com/datasets/anandaramg/global-superstore

In [11]:
import kagglehub, os, pathlib

DATASET_REF = "vivek468/superstore-dataset-final"   # you can change this to any public dataset reference

download_path = kagglehub.dataset_download(DATASET_REF)
print("Path to dataset files:", download_path)

# List files
p = pathlib.Path(download_path)
print("Files:")
for f in p.glob("**/*"):
    if f.is_file():
        print("-", f)

Using Colab cache for faster access to the 'superstore-dataset-final' dataset.
Path to dataset files: /kaggle/input/superstore-dataset-final
Files:
- /kaggle/input/superstore-dataset-final/Sample - Superstore.csv


## 3) Prompting approach for Lab 1
You'll **paste prompts into Gemini** (in a separate tab), get back **SQL**, then paste that SQL here to execute.  
We’ll practice three core question types:
- **“What”** (lists & filters) → `SELECT`, `WHERE`, `DISTINCT`
- **“How many”** (counts by category) → `COUNT`, `GROUP BY`
- **“Who is best”** (rank/limit by metric) → `SUM`, `ORDER BY`, `LIMIT`

### 3A) “What” Question — `SELECT`, `WHERE`, `DISTINCT`
**Business Question:** *A manager wants a list of all **unique** product sub‑categories sold in the **West** region.*

**Paste this prompt into Gemini (no edits needed except the project id if you used a different table path):**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Find all the unique values in the 'Sub_Category' column, but only for rows where the 'Region' column is exactly 'West'.
```

**Then paste the SQL you get into the cell below** (replace the placeholder) and run it.

In [13]:
sql_str = """
SELECT DISTINCT Sub_Category
FROM `imposing-coast-442802-a7.superstore_data.sales`
WHERE Region = 'West'
ORDER BY Sub_Category
"""

df_what = bq.query(sql_str).to_dataframe()
print(f"Rows: {len(df_what)}")
display(df_what.head(20))

Rows: 17


,Sub_Category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


**Explain it back (metacognition):** In Gemini, paste:  
> *Explain the following SQL query line by line: [paste your SQL]*  
Summarize what you learned here:

> **Notes:**

### 3B) “How many” Question — `COUNT`, `GROUP BY`
**Business Question:** *How many orders were placed in each **Ship Mode**?*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`.
# GOAL: Count the total number of records for each unique value in the 'Ship_Mode' column.
# The final result should have two columns: 'Ship_Mode' and 'order_count'.
```


In [14]:
sql_str = """
SELECT Ship_Mode, COUNT(*) AS order_count
FROM `imposing-coast-442802-a7.superstore_data.sales`
GROUP BY Ship_Mode
ORDER BY order_count DESC
"""

df_howmany = bq.query(sql_str).to_dataframe()
display(df_howmany)

,Ship_Mode,order_count
0,Standard Class,5968
1,Second Class,1945
2,First Class,1538
3,Same Day,543


### 3C) “Who is best” Question — `SUM`, `ORDER BY`, `LIMIT`
**Business Question:** *Identify the **top 5 most profitable customers**.*

**Gemini prompt:**
```
# TASK: Generate a BigQuery SQL query.
# CONTEXT: The table is `[YOUR_PROJECT_ID].superstore_data.sales`. The customer identifier is 'Customer_ID'.
# GOAL: Calculate the sum of 'Profit' for each customer. The final output should show the 'Customer_ID' and their total profit, sorted from highest to lowest profit, and limited to only the top 5 results.
```


In [17]:
sql_str = """
SELECT Customer_ID, SUM(Profit) AS total_profit
FROM `imposing-coast-442802-a7.superstore_data.sales`
GROUP BY Customer_ID
ORDER BY total_profit DESC
LIMIT 5
"""

df_best = bq.query(sql_str).to_dataframe()
display(df_best)

,Customer_ID,total_profit
0,TC-20980,8981.3239
1,RB-19360,6976.0959
2,SC-20095,5757.4119
3,HL-15040,5622.4292
4,AB-10105,5444.8055


## 4) Challenge prompts (author your own)
Write **your own precise prompt** in a text cell (or comment) for each question below, then get SQL from Gemini and run it in the provided code cells.

**Challenge 1:** *What is the **average discount** for products in the **Technology** category sold in the **East** region?*  
**Challenge 2:** *How many **unique customers** has each **Segment** (Consumer/Corporate/Home Office) served?*

In [18]:
sql_str = """
SELECT AVG(Discount) AS avg_discount
FROM `imposing-coast-442802-a7.superstore_data.sales`
WHERE Category = 'Technology' AND Region = 'East'
"""
df_ch1 = bq.query(sql_str).to_dataframe()
display(df_ch1)

,avg_discount
0,0.143364


In [19]:
sql_str = """
SELECT Segment, COUNT(DISTINCT Customer_ID) AS unique_customers
FROM `imposing-coast-442802-a7.superstore_data.sales`
GROUP BY Segment
ORDER BY unique_customers DESC
"""

df_ch2 = bq.query(sql_str).to_dataframe()
display(df_ch2)

,Segment,unique_customers
0,Consumer,409
1,Corporate,236
2,Home Office,148


## 5) Reflection (DIVE mindset)
- **Discover:** What did you find first?  
- **Investigate:** What alternate query or filter changed the story?  
- **Validate:** Where could the AI‑generated SQL be wrong or incomplete? How did you check?  
- **Extend:** Which stakeholder could use your results tomorrow? What action should they take?

## 6) Save your work to GitHub (pick one of the options)
**Option A (recommended):** In Colab, go to **File → Save a copy in GitHub…** and select your team repo + folder (e.g., `labs/Unit1/`).  
**Option B (CLI, if you know git):**
```bash
# (In Colab) mount Drive, then clone/pull/push as usual with a PAT
# Be careful to NOT store secrets in the notebook.
```
Name the file **`Lab1_AI_Assisted_SQL.ipynb`** and push it to your team repo.